In [1]:
import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential,load_model,Model
from tensorflow.python.keras.layers import Dense,Activation,Dropout,Conv2D,AvgPool2D,Flatten,MaxPool2D,BatchNormalization,GlobalAvgPool2D
from tensorflow.python.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.python.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications import MobileNetV2,InceptionResNetV2
from tensorflow.python.keras.applications.resnet import ResNet101,ResNet152,ResNet50
from tensorflow.python.keras.applications import VGG16,VGG19
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import to_categorical

In [2]:
(train_x,train_y),(test_x,test_y)=mnist.load_data()

In [3]:
print("The size of train_x is: {}".format(train_x.shape))
print("The size of train_y is: {}".format(train_y.shape))

print("The size of test_x is: {}".format(test_x.shape))
print("The size of test_y is: {}".format(test_y.shape))

The size of train_x is: (60000, 28, 28)
The size of train_y is: (60000,)
The size of test_x is: (10000, 28, 28)
The size of test_y is: (10000,)


# Reshaping the Image values

In [4]:
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
print(train_x.shape)
print(test_x.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


# Normalizing the values of the Image

In [5]:
train_x = train_x/255.0
test_x = test_x/255.0

# One Hot Encoding The lables

In [6]:
train_y = to_categorical(train_y, num_classes=10)
test_y = to_categorical(test_y, num_classes=10)
print(train_y.shape)
print(test_y.shape)

(60000, 10)
(10000, 10)


In [7]:
# Creating a Con

In [8]:
model = Sequential()

# Adding a Convolution Layer C1
# Input shape = N = (28 x 28)
# No. of filters  = 6
# Filter size = f = (5 x 5)
# Padding = P = 0
# Strides = S = 1
# Size of each feature map in C1 is (N-f+2P)/S +1 = 28-5+1 = 24
# No. of parameters between input layer and C1 = (5*5 + 1)*6 = 156
model.add(Conv2D(filters=6, kernel_size=(5,5), padding='valid', input_shape=(28,28,1), activation='tanh'))

# Adding an Average Pooling Layer S2
# Input shape = N = (24 x 24)
# No. of filters = 6
# Filter size = f = (2 x 2)
# Padding = P = 0
# Strides = S = 2
# Size of each feature map in S2 is (N-f+2P)/S +1 = (24-2+0)/2+1 = 11+1 = 12
# No. of parameters between C1 and S2 = (1+1)*6 = 12
model.add(AvgPool2D(pool_size=(2,2)))

# Adding a Convolution Layer C3
# Input shape = N = (12 x 12)
# No. of filters  = 16
# Filter size = f = (5 x 5)
# Padding = P = 0
# Strides = S = 1
# Size of each feature map in C3 is (N-f+2P)/S +1 = 12-5+1 = 8
# No. of parameters between S2 and C3 = (5*5*6*16 + 16) + 16 = 2416
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))

# Adding an Average Pooling Layer S4
# Input shape = N = (8 x 8)
# No. of filters = 16
# Filter size = f = (2 x 2)
# Padding = P = 0
# Strides = S = 2
# Size of each feature map in S4 is (N-f+2P)/S +1 = (8-2+0)/2+1 = 3+1 = 4
# No. of parameters between C3 and S4 = (1+1)*16 = 32
model.add(AvgPool2D(pool_size=(2,2)))

# As compared to LeNet-5 architecture there was one more application of convolution but in our code  further application of 
# convolution with (5 x 5) filter would result in a negative dimension which is not possible. So we aren't applying any more
# convolution here.

# Flattening the layer S4
# There would be 16*(4*4) = 256 neurons
model.add(Flatten())

# Adding a Dense layer with `tanh` activation+# 
# No. of inputs = 256
# No. of outputs = 120
# No. of parameters = 256*120 + 120 = 30,840
model.add(Dense(120, activation='tanh'))

# Adding a Dense layer with `tanh` activation
# No. of inputs = 120
# No. of outputs = 84
# No. of parameters = 120*84 + 84 = 10,164
model.add(Dense(84, activation='tanh'))

# Adding a Dense layer with `softmax` activation
# No. of inputs = 84
# No. of outputs = 10
# No. of parameters = 84*10 + 10 = 850
model.add(Dense(10, activation='softmax'))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________

In [9]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=128, epochs=20, verbose=1, validation_data=(test_x, test_y))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 33s 545us/sample - loss: 0.3518 - acc: 0.8996 - val_loss: 0.1617 - val_acc: 0.9506
Epoch 2/20
60000/60000 [==============================] - 33s 554us/sample - loss: 0.1346 - acc: 0.9581 - val_loss: 0.1035 - val_acc: 0.9682
Epoch 3/20
60000/60000 [==============================] - 33s 546us/sample - loss: 0.0899 - acc: 0.9726 - val_loss: 0.0742 - val_acc: 0.9769
Epoch 4/20
60000/60000 [==============================] - 33s 551us/sample - loss: 0.0668 - acc: 0.9796 - val_loss: 0.0629 - val_acc: 0.9808
Epoch 5/20
60000/60000 [==============================] - 32s 528us/sample - loss: 0.0525 - acc: 0.9839 - val_loss: 0.0609 - val_acc: 0.9801
Epoch 6/20
60000/60000 [==============================] - 32s 531us/sample - loss: 0.0439 - acc: 0.9866 - val_loss: 0.0539 - val_acc: 0.9823
Epoch 7/20
60000/60000 [==============================] - 32s 527us/sample - loss: 0.0380 - acc: 0.9883 

In [10]:
score = model.evaluate(test_x, test_y)
trainscore=model.evaluate(train_x,train_y)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])
print("Train Loss",trainscore[0])
print("Train accuracy",trainscore[1])

60000/60000 [==============================] - 6s 107us/sample - loss: 0.0066 - acc: 0.9982
Test Loss: 0.04516613108418678
Test accuracy: 0.9876
Train Loss 0.006585052784008925
Train accuracy 0.9981667


In [11]:
model.save("my_model.h5")
later_model=load_model("my_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
